# IG360 QA - Create Multi-Profile
*Purpose:* Create a short profile for multiple users:
<ul>
    <li>User name, full name</li>
    <li>Stats on followers, following</li>
    <li>Date of most recent post</li>
</ul>
<br>
<br>
For this example we will look at users that target account follows though they do not follow back

In [1]:
# import IG360 data
from IG360.ig360_scrape import *

# import other key libraries
import pandas as pd
import datetime
import json
import re
from pathlib import Path

# set output folder
txt_dir_output = '_output/'
dir_output = Path(txt_dir_output)

# locate configuration file
config_file_path = 'config.ini'
config = init_config(config_file_path)

# set user and password (turned off by default; using config.ini)
#user_login = 'xx'
#user_password = 'xx'

# set target account and capture parameters
tgt_user = "worldofbenches"

Initialize Scraper and Login

In [2]:
#scraper = IG360Scrape(config, user_login, user_password)
scraper = IG360Scrape(config)
scraper.login()

Get list of every account following or being followed by user

In [3]:
# extract follower and following data
scraper.scrape_followers(tgt_user)
scraper.scrape_following(tgt_user)

# dump partitioned follow relationship data
part_users = scraper.get_user_part_lists()
followers_only = part_users[0]
following_only = part_users[1]
mutual_follow = part_users[2]

For each non-reciprocating account, combine short profile with date of most recent post

In [4]:
profiles = []

for usr in followers_only:
    # get profile info
    scraper.scrape_profile(usr)
    profile_data = scraper.get_profile_data()
    # get post data
    scraper.scrape_post_list(usr, 1)
    post_list = scraper.get_posts()
    if len(post_list) > 0:
        scraper.scrape_post(post_list[0].split('/')[-2], get_likes=False)
        post_data = scraper.get_post_data()
        # combine and append
        profiles.append(list(profile_data) + [post_data[1]])
    else:
        profiles.append(list(profile_data) + [None])


Process Final Export; Output to JSON

In [5]:
scraper.util_json_write_mp(tgt_user, dir_output, profiles)

In [6]:
profiles

[['thesodatruckut',
  0,
  202,
  989,
  555,
  'The Soda Truck',
  'Serving up flavored sodas, house-made specialties and sweets. Seasonal hot chocolate. Book us for your event at events@thesodatruck.co',
  '2019-10-26T13:48:15.000Z'],
 ['morganvalleylamb',
  0,
  36,
  135,
  91,
  'Morgan Valley Lamb',
  'Find choice cuts at delicious, local restaurants or online!',
  '2018-04-16T20:11:03.000Z'],
 ['rossi_lew97',
  0,
  0,
  13,
  29,
  'Ross Lewis',
  '24, country boy, Copenhagen and Baseball are life.💯',
  None],
 ['xjustxmikex',
  0,
  386,
  550,
  1365,
  'Michael Soliz',
  'Queen of Summertime',
  '2020-03-23T17:13:00.000Z'],
 ['zubairu.aminu',
  0,
  20,
  422,
  1640,
  'Aminu Zubairu',
  '',
  '2018-03-20T05:15:41.000Z'],
 ['thedaysof47',
  0,
  46,
  279,
  1056,
  'Days Of 47',
  "Parade, Rodeo, and free family fun! Join us for all the Days of '47 events this summer 🇺🇸💥Follow us also on Facebook and Twitter #daysof47 #pioneerday",
  '2018-07-24T23:03:47.000Z'],
 ['monkeyb